In [1]:
# Tile level classifier - Tiles generated from WSI/SVS slides of TCGA dataset
# A multi classes image classifier, based on convolutional neural network using Keras and Tensorflow. 
# A multi-label classifier (having one fully-connected layer at the end), with multi-classification (10 classes, in this instance)
# Largely copied from the code https://github.com/kallooa/MSDA_Capstone_Final/tree/master/3_Model_Training/Tile_Level_Model_Training
# classifying 10 cancer types (acc, blca, brca, cesc, coad, chol, dlbc, lgg, gbm and read tiles generated from around 400-1200 slides (each class)
# with each slide having around 270,000 tiles) downloaded from candygram.emory.edu
# Thumbnail extraction and tile extraction from WSI slides are implemented using HistomicsTK (https://girder.readthedocs.io/en/latest/)
# Used Keras.ImageDataGenerator for Training/Validation data augmentation and the augmented images are flown from respective directory
# train folder has training images and test folder has test images
# val folder has validation images, autogenerated by keras 
# Environment: A docker container having Keras, TensorFlow, Python-3 with GPU based execution

# 1024 - FCNs

In [2]:
import matplotlib
matplotlib.use('Agg')
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing import image
from keras.models import Model, load_model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Convolution2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.callbacks import Callback
from keras.optimizers import adagrad, adadelta, rmsprop, adam
from keras.utils import np_utils
from keras.regularizers import l2
from sklearn.cross_validation import StratifiedKFold
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
import datetime, time, os, sys
import numpy as np
import h5py
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support
import pandas as pd
from log_utils import setup_logging_to_file, log_exception # catch and logging exceptions

Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
sys.setrecursionlimit(10000)

# User Input:¶

In [4]:
# Get timestamp:
now = datetime.datetime.now()
filetime = str(now.year)+str(now.month)+str(now.day)+'_'+str(now.hour)+str(now.minute)

# Logging to a file
log_file = "v3_1024FCN_10_class_"+ filetime +".log"
setup_logging_to_file(log_file)
f = open(log_file, "a")

# User Input
# Image dimension:
img_width, img_height = 150,150
# Epochs
epochs = 2
# Batch size:
batch_size = 25
colormode ="rgb"
channels=3

In [5]:
# Training and Validation Images Locations
training_dir = '/data/train'
test_dir = '/data/test'

# Results Location:
results_dir ="/data/results/"

# Basic Image Statistics:¶

In [6]:
# Count training images:
ntraining = 0
for root, dirs, files in os.walk(training_dir):
    ntraining += len(files)

# Getting data format:

In [7]:
# get data format:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

# class_weights for unbalanced data

In [22]:
# Defining class_weights for unbalanced classe
class_weight = {0:2.64, 1:1., 2:2.27, 3:1.73, 4:6.79, 5:5.40, 6:5.69, 7:2.50, 8:1.71, 9:6.21}
#class_weight = {'acc':2.64, 'blca':1., 'brca':2.27, 'cesc':1.73, 'chol':6.79, 'coad':5.40, 'dlbc':5.69, 'gbm':2.50, 'lgg':1.71, 'read':6.21}

# Data Augmentation:¶

In [23]:
# Data Augmentation
# Training Data Generator with Augmentation:
# -Scale
# -Shear
# -Zoom
# -Height and Width Shift
# -Fill: Nearest
# -Horizontal Flip
train_datagen = ImageDataGenerator(vertical_flip=True, horizontal_flip=True, validation_split=0.2)

# Validation Data Generator:
val_datagen = ImageDataGenerator(validation_split=0.2)

# Training Data flow from directory:
train_generator = train_datagen.flow_from_directory(
    training_dir,
    target_size=(150,150),
    batch_size=batch_size,
    color_mode=colormode,
    subset='training')

# Validation Data flow from directory:
val_generator = val_datagen.flow_from_directory(
    training_dir, 
    target_size=(150,150),
    batch_size=batch_size,
    color_mode=colormode,
    subset='validation')

# Number of Classes/Labels:
nLabels = len(val_generator.class_indices)

Found 119530 images belonging to 10 classes.
Found 29878 images belonging to 10 classes.


In [24]:
val_generator.class_indices

{'acc': 0,
 'blca': 1,
 'brca': 2,
 'cesc': 3,
 'chol': 4,
 'coad': 5,
 'dlbc': 6,
 'gbm': 7,
 'lgg': 8,
 'read': 9}

# Model¶

In [25]:
# Model
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer
predictions = Dense(nLabels, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = True

for layer in model.layers:
    layer.trainable = True

# ompile the model (should be done *after* setting layers to non-trainable)
# create model with for binary output with the adam optimization algorithm
model.compile(optimizer='adadelta', loss='categorical_crossentropy',  metrics=['accuracy'])

In [26]:
# To avoid broken stream or IO error for >=100k images
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Model Run

In [ ]:
# Model fitting and training run
start_time = datetime.datetime.now()
print >>f, "Training Start Time:%s " % start_time 
inception_10_tile_class_model=model.fit_generator(train_generator, steps_per_epoch= ntraining, class_weight=class_weight,epochs=epochs, \
    validation_data=val_generator,use_multiprocessing=True, workers=16, max_queue_size=32)

end_time = datetime.datetime.now()
print >>f, "Training took %s to finish %s epochs" % ((end_time - start_time), epochs)

Epoch 1/2
  8078/149408 [>.............................] - ETA: 11:16:53 - loss: 2.5557 - acc: 0.5760

In [ ]:
! mkdir -p /output/results/

# Save Model into a File

In [ ]:
# Save timestamped model to modelfilename
modelfilename='10_tile_class_inception_model_'+filetime+'.h5'
model.save(modelfilename)

# Save model into .csv file
hist = inception_10_tile_class_model.history
hist = pd.DataFrame(hist)
hist.to_csv('10_tile_class_inception_model_'+filetime+'.csv')

# Run Model on Test Images:

In [ ]:
# run model on Test Images
nTest = 0
for root, dirs, files in os.walk(test_dir):
    nTest += len(files)

In [ ]:
# Testing Data Generator:
# modified to suit histology images - 17th Aug, 2018
#test_datagen = ImageDataGenerator(rescale=1. /255.)
#testing_generator_noShuffle = test_datagen.flow_from_directory(
#    test_dir,
#    target_size=(img_width, img_height),
#    batch_size=batch_size,
#    shuffle=False,
#    class_mode='categorical')

test_datagen = ImageDataGenerator()
testing_generator_noShuffle = test_datagen.flow_from_directory(
    test_dir, 
    target_size=(150,150),
    batch_size=batch_size,
    color_mode=colormode,
    shuffle = True,
    class_mode='categorical')

In [ ]:
# predict_Validation: narray
# row= image
# column= probability of falling within label matching column_index
predict_Testing = model.predict_generator(testing_generator_noShuffle)

In [ ]:
# Best Prediction for all labels
best_prediction_per_label= [ max( predict_Testing[:,j] ) for j in range( predict_Testing.shape[1] ) ]

# Predicted label for each image:

In [ ]:
predicted_labels= []
# Find highest probability in prediction list for each image
for i in predict_Testing:
    i= list(i)
    max_value = max(i) 
    predicted_labels.append( i.index(max_value) )

# Confusion Matrix generation

In [ ]:
# Confusion Matrix in a Plot
import matplotlib.pyplot as pyplot
pyplot.figure
target_names = testing_generator_noShuffle.class_indices
pyplot.figure(figsize=(10,10))
cnf_matrix = confusion_matrix(testing_generator_noShuffle.classes, predicted_labels)
classes = list(target_names)
pyplot.imshow(cnf_matrix, interpolation='nearest')
pyplot.colorbar()
tick_marks = np.arange(len(classes))  
_ = pyplot.xticks(tick_marks, classes, rotation=90)
_ = pyplot.yticks(tick_marks, classes)
plotopt= 'inception_10_tile_class_model_'+filetime+'.png'
pyplot.savefig(plotopt)

In [ ]:
cls_rpt = classification_report(testing_generator_noShuffle.classes, predicted_labels, target_names= testing_generator_noShuffle.class_indices)

In [ ]:
print(cls_rpt)

# Turning into classification report into classification object

In [ ]:
avgresults = cls_rpt.strip().split('\n')[-1].split()

In [ ]:
overallResults={'label' : 'avg/total', 'precision': avgresults[3], 'recall':avgresults[4],'f1-score':avgresults[5], 'support':avgresults[6]}

In [ ]:
precision, recall, fscore, support  =  precision_recall_fscore_support(testing_generator_noShuffle.classes, predicted_labels)

# Create Prediction object for each test image with filename, actual image label, Folder

In [ ]:
import glob
import os
import keras.preprocessing.image as Kimg

modelInfo['imgprediction'] = []

model=load_model(modelfilename)

for fld in os.listdir('/data/test/'): 
    trueLabel = fld
    for img in os.listdir('/data/test/%s/' %trueLabel): 
        imgPath = "/data/test/%s/%s" % (fld, img)
        x = Kimg.load_img(imgPath, target_size=(64,64))
        x = Kimg.img_to_array(x)
        x = x.reshape((1,) + x.shape)
        x = x/255.
        pr=model.predict(x)
        curr = {'filename': img, 'actualImageLabel': fld, 'modelprediction':pr} 
        modelInfo['imgprediction'].append(curr) 

In [ ]:
modelInfo['imgprediction']

# Generates Top3 Predicted images for each individual image in the "test folder" only

In [ ]:
from PIL import Image
import matplotlib.pyplot as pyplot
rows = 2
cols = 2
img_width, img_height = 64, 64
pyplot.figure()
pyplot.show()
fig, ax = pyplot.subplots(rows, cols, frameon=False, figsize=(5, 5))
fig.suptitle('Prediction Images', fontsize=10, y = 1.03)
count=0

modelInfo['imgprediction'] = []

model=load_model(modelfilename)

for fld in os.listdir('/data/test/'): 
    trueLabel = fld          
    for imgname in os.listdir('/data/test/%s/' %trueLabel): 
        for i in range(rows):
            for j in range(cols):
                imgPath = "/data/test/%s/%s" % (fld, imgname)                
                img = Image.open(imgPath)
                img = img.resize((img_width, img_height), Image.ANTIALIAS)
                ax[i][j].imshow(img)
                img = img_to_array(img)
                img = img/255.0
                img = img.reshape((1,) + img.shape)
                pr = model.predict(img, batch_size= 1)                
                curr = {'filename': img, 'actualImageLabel': fld, 'modelprediction':pr} 
                modelInfo['imgprediction'].append(curr)                
                # To show image with top 3 predicted images
                pred = pd.DataFrame(np.transpose(np.round(pr, decimals = 3)))
                pred = pred.nlargest(n = 3, columns = 0) 
                pred['char'] = [list(modelInfo['labelname_to_index'].keys())[list(modelInfo['labelname_to_index'].values()).index(x)] for x in pred.index]
                charstr = ''
                for k in range(0,3):
                    if k < 2:
                        charstr = charstr+str(pred.iloc[k,1])+': '+str(pred.iloc[k,0])+'\n'
                    else:
                        charstr = charstr+str(pred.iloc[k,1])+': '+str(pred.iloc[k,0])                
                ec = (0, .8, .1)
                fc = (0, .9, .2)
                count = count + 1
                ax[i][j].text(0, -10, charstr, size=10, rotation=0,
                      ha="left", va="top", 
                      bbox=dict(boxstyle="round", ec=ec, fc=fc, alpha = 0.7))
                pyplot.setp(ax, xticks=[], yticks=[])
                pyplot.tight_layout(rect=[0, 0.14, 1, 0.95])
                pyplot.savefig('/data/code/results/prediction_'+ str(imgname) + '.png')